In [1]:
import polars as pl
import json

In [5]:
with open("hot100_datafiles/records05-29_22-36.json", "r") as f:
    data = json.load(f)


In [6]:
df = pl.DataFrame(data)

In [7]:
df

position,date,song,artist,wks_on_chart
i64,str,str,str,str
1,"""2025-04-19""","""1""","""Luther""","""1"""
2,"""2025-04-19""","""2""","""Nokia""","""2"""
3,"""2025-04-19""","""3""","""Die With A Smile""","""1"""
4,"""2025-04-19""","""4""","""All The Way""","""4"""
5,"""2025-04-19""","""5""","""Pink Pony Club""","""5"""
…,…,…,…,…
96,"""1958-11-29""","""96""","""Almost In Your Arms""","""78"""
97,"""1958-11-29""","""97""","""Cinderella""","""91"""
98,"""1958-11-29""","""98""","""Sweetheart""","""98"""


In [15]:
ranked_songs = df.groupby(["song", "artist"])["position"].apply(lambda x: (1/x).sum())

In [16]:
ranked_songs = ranked_songs.sort_values(ascending=False)


In [17]:
ranked_songs = ranked_songs.reset_index(name="popularity_score")
df_alltime = pd.DataFrame(ranked_songs)

In [20]:
floor = df_alltime["popularity_score"].min()
ceiling = df_alltime["popularity_score"].max()

In [25]:
stats = df_alltime["popularity_score"].describe()
stats

count    26093.000000
mean         0.196416
std          0.569664
min          0.010000
25%          0.022858
50%          0.055059
75%          0.146873
max         26.972293
Name: popularity_score, dtype: float64

In [29]:
percentile_popularity = df_alltime["popularity_score"].rank(pct=True)
df_alltime["percentile"] = percentile_popularity

In [ ]:
df_alltime # name index song_id 

,song,artist,popularity_score,percentile
0,A Bar Song (Tipsy),Shaboozey,26.972293,1.000000
1,Die With A Smile,Lady Gaga & Bruno Mars,16.079458,0.999962
2,As It Was,Harry Styles,15.577342,0.999923
3,Luther,Kendrick Lamar & SZA,15.506518,0.999885
4,I Had Some Help,Post Malone Featuring Morgan Wallen,15.242586,0.999847
...,...,...,...,...
26088,Say OK,Vanessa Hudgens,0.010000,0.004158
26089,Raise Your Weapon,deadmau5 Featuring Greta Svabo Bech,0.010000,0.004158
26090,One Sunny Day (From Quicksilver),"Ray Parker,Jr. & Helen Terry",0.010000,0.004158
26091,"The More You Live, The More You Love",A Flock Of Seagulls,0.010000,0.004158


In [ ]:
test = df.groupby("song")["date"].agg(["min", "max"])
test # add record_id to main df, normalize tables, song table with song_id etc

,min,max
song,,
"""B"" Girls",1990-07-14 22:45:02.677242,1990-07-14 22:45:02.677242
"""Cherry Cherry"" from Hot August Night",1973-03-17 22:45:02.677242,1973-05-12 22:45:02.677242
"""Having A Party"" Medley",1973-10-13 22:45:02.677242,1973-12-08 22:45:02.677242
"""Joy"" Pt. I",1974-01-12 22:45:02.677242,1974-02-02 22:45:02.677242
"""Yep!""",1959-04-04 22:45:02.677242,1959-05-16 22:45:02.677242
...,...,...
thanK you aIMee,2024-05-04 22:45:02.677242,2024-05-18 22:45:02.677242
the.climb.back,2021-05-29 22:45:02.677242,2021-05-29 22:45:02.677242
whoknows,2004-04-24 22:45:02.677242,2004-06-12 22:45:02.677242
